In [ ]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df_full = pd.read_csv('used_clean.csv')

# Review the DataFrame
df_full.head()

In [ ]:
df_full.info()

In [ ]:
col = ['price', 'miles', 'year', 'make', 'model', 'body_type',
       'vehicle_type', 'drivetrain', 'transmission', 'fuel_type',
       'engine_size', 'engine_block', 'state']

df = df_full[col].copy()

In [ ]:
df["engine_size"] = df["engine_size"].apply(lambda x: 0 if x == 'E' else x)
df.head()

In [ ]:
df.engine_size = df.engine_size.astype(float)

In [ ]:
def label_engine_size(engine):
    if 0.1 <= engine < 2:
        return 1
    elif 2 <= engine < 3:
        return 2
    elif 3 <= engine < 4:
        return 3
    elif 4 <= engine < 5:
        return 4
    elif 5 <= engine < 6:
        return 5
    elif 6 <= engine < 7:
        return 6
    elif 7 <= engine < 8:
        return 7
    else:
        return 0

# Apply the custom function to the 'fuel_type' column
df['engine_size'] = df['engine_size'].apply(label_engine_size)

# Show the modified DataFrame
df.head(10)

In [ ]:
cutoff = 1000
application_types_to_replace = df.model.value_counts()[df.model.value_counts() < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['model'] = df['model'].replace(app,"Other")

# Check to make sure replacement was successful
df['model'].value_counts()

In [ ]:
#cutoff = 500
#application_types_to_replace = df.trim.value_counts()[df.trim.value_counts() < cutoff].index.tolist()

# Replace in dataframe
#for app in application_types_to_replace:
    #df['trim'] = df['trim'].replace(app,"Other")

# Check to make sure replacement was successful
#df['trim'].value_counts()

In [ ]:
def label_fuel_type(fuel):
    fuel = fuel.lower()  # Convert to lowercase for consistent matching
    if 'diesel' in fuel:
        return 'Diesel'
    elif 'electric' in fuel and '/' in fuel:
        return 'Hybrid'
    elif fuel == 'electric':
        return 'Electric'
    else:
        return 'Gas'

# Apply the custom function to the 'fuel_type' column
df['fuel_type'] = df['fuel_type'].apply(label_fuel_type)

# Show the modified DataFrame
df.head(10)

In [ ]:
def label_body_type(body):
    if 'Car Van'  in body or 'Combi' in body:
        return 'Cargo Van'
    elif 'Targa' in body or 'Roadster' in body:
        return 'Coupe'
    elif 'Cutaway' in body or 'Chassis Cab' in body:
        return 'Pickup'
    elif 'Targa' in body:
        return 'Roadster'
    elif 'Mini Mpv' in body:
        return 'Hatchback'
    elif 'Passenger Van' in body:
        return 'Minivan'
    else:
        return body

# Apply the custom function to the 'fuel_type' column
df['body_type'] = df['body_type'].apply(label_body_type)

# Show the modified DataFrame
df.head(10)

In [ ]:
# Define Preprocessing Pipelines

# Define preprocessing for numeric features (Age, Fare, Family_Size) (note that for age, we just use the column median, not split by gender)
numeric_features = ['miles', 'year', 'engine_size'] # You can also do this in a loop, select the numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for the binary features (Sex, Has_Cabin)
binary_features = ['vehicle_type', 'transmission']
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),
    ('label', OrdinalEncoder())])  # Label encode for binary feature

# Define preprocessing for categorical features (Pclass, Embarked) 
categorical_features = ['make', 'model', 'body_type', 'drivetrain', 'fuel_type', 'engine_block', 'state']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing for numeric and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('binary', binary_transformer, binary_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# Correlation Analysis (this is optional and not needed for the ML Experiment)
# It just shows what the data looks like after transformation before training
# We will still declare a full pipeline of preprocessing + training

# Use only preprocessing pipeline to transform the data
preprocessed_X_train = preprocessor.fit_transform(df)

# Convert preprocessed data to a DataFrame
# Get the feature names after one-hot encoding
encoded_feature_names = (numeric_features + binary_features +
                         list(preprocessor.transformers_[2][1]['onehot'].get_feature_names_out(categorical_features)))

df_final = pd.DataFrame(preprocessed_X_train, columns=encoded_feature_names)
df_final["price"] = df.price
df_final.head()

In [ ]:
# Step 1: Get the data (NOTE: this is the original dataframe, not the one used for the correlations)
X = df.drop(columns=["price"])
y = df.price

y = np.log(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1) # stratify=True maintains target class percentages
print(X_train.shape, X_test.shape)

In [ ]:
# Function for Regression
def doRegression(model, X_train, X_test, y_train, y_test):
    # Step 3: Fit the model
    model.fit(X_train, y_train)
    
    # Step 4: Evaluate the model
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    # Generate metrics TRAIN
    train_r2 = r2_score(y_train, train_preds)
    train_mse = mean_squared_error(y_train, train_preds)
    train_mae = mean_absolute_error(y_train, train_preds)
    train_rmse = np.sqrt(train_mse)
    
    train_results = f"""TRAIN METRICS
    R2: {train_r2}
    MSE: {train_mse}
    RMSE: {train_rmse}
    MAE: {train_mae}
    """
    
    print(train_results)

    # Generate metrics TEST
    test_r2 = r2_score(y_test, test_preds)
    test_mse = mean_squared_error(y_test, test_preds)
    test_mae = mean_absolute_error(y_test, test_preds)
    test_rmse = np.sqrt(test_mse)
    
    test_results = f"""TEST METRICS
    R2: {test_r2}
    MSE: {test_mse}
    RMSE: {test_rmse}
    MAE: {test_mae}
    """
    
    print(test_results)

    # VISUALIZE TEST RESULTS
    # Predicted vs Actual Plot
    plt.scatter(y_test, test_preds)
    plt.plot(y_test, y_test)
    plt.title("Predicted vs Actual Plot")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.show()

    # Residual Plot
    resids = test_preds - y_test
    plt.scatter(test_preds, resids)
    plt.hlines(0, min(test_preds), max(test_preds))
    plt.title("Residual Plot")
    plt.xlabel("Predictions")
    plt.ylabel("Residuals")
    plt.show()

In [ ]:
# Step 2: Init the Model
rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestRegressor(random_state=42))
])

# Do Machine Learning
doRegression(rf, X_train, X_test, y_train, y_test)

In [ ]:
# Create a pipeline with a scaler and a logistic regression model
rfr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestRegressor(random_state=42))
])

# Perform 5-fold cross-validation with roc_auc scoring

cv = KFold(n_splits=5)  # For regression, KFold is used instead of StratifiedKFold
cv_scores = cross_val_score(rfr, X, y, cv=cv, scoring='neg_mean_squared_error')

# Output the results (MSE, so we negate the score to make it positive)
mse_scores = -cv_scores  # Cross_val_score returns negative values for MSE when using 'neg_mean_squared_error'
print(f"Cross-validation MSE scores: {mse_scores}")
print(f"Mean MSE: {np.mean(mse_scores):.3f}")
print(f"Mean RMSE: {np.sqrt(np.mean(mse_scores)):.3f}")  # Root Mean Squared Error (RMSE) is a more interpretable metric

In [ ]:
# Retrain the model on the ENTIRE dataset
rf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestRegressor(random_state=42))
])
rf.fit(X, y)